In [1]:
import datastore

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

spot = datastore.getSpotPrice()

cons_gwh = datastore.getConsumptionGWh()

nuclear_finland_germany_mwh = datastore.getHourlyProduction_nuclear_EU()
germany_all = datastore.getHourlyProduction_Germany_all()

production_se1_mwh = datastore.getAllSE1ProductionMWh() 
production_se2_mwh = datastore.getAllSE2ProductionMWh()
production_se3_mwh = datastore.getAllSE3ProductionMWh()
production_se4_mwh = datastore.getAllSE4ProductionMWh()

hydroReservoirs = datastore.getWeeklyHydroReservs()

temp_pen = datastore.getDailyWeather()

# stationCap = datastore.getStationCap_forecast()
# transTo = datastore.getTransmissionCapTo_forecast()
# transFrom = datastore.getTransmissionCapFrom_forecast()

# productionCap = datastore.getProductionCap_forecast()

Missing values in Index(['spotPrice'], dtype='object') : 6
Missing values in Index(['ConsumptionSE1GWh', 'ConsumptionSE2GWh', 'ConsumptionSE3GWh',
       'ConsumptionSE4GWh'],
      dtype='object') : 24
Missing values in Index(['nucFI(MWh)', 'nucDE(MWh)'], dtype='object') : 1224
Missing values in Index(['wind(Mwh)', 'solar(Mwh)', 'trm(Mwh)', 'biomass(Mwh)', 'coal(MWh)',
       'gas(MWh)', 'oil(MWh)', 'lignite(MWh)', 'nuc(MWh)', 'pump(MWh)',
       'other(MWh)', 'waste(MWh)'],
      dtype='object') : 1537
Missing values in Index(['waterSE1(MWh)', 'windSE1(MWh)', 'trmSE1MWh'], dtype='object') : 94
Missing values in Index(['waterSE2(MWh)', 'windSE2(MWh)', 'solSE2MWh', 'trmSE2MWh', 'OthSE2MWh'], dtype='object') : 240
Missing values in Index(['waterSE3(MWh)', 'windSE3(MWh)', 'nucSE3(MWh)', 'solSE3MWh',
       'trmSE3MWh', 'OthSE3MWh'],
      dtype='object') : 259
Missing values in Index(['waterSE4(MWh)', 'windSE4(MWh)', 'solSE4MWh', 'trmSE4MWh', 'OthSE4MWh'], dtype='object') : 251
Missing v

In [2]:
import calendar_features as cf

dataset = pd.concat([spot, cons_gwh, production_se1_mwh, production_se2_mwh, 
                                    production_se3_mwh, production_se4_mwh, temp_pen, hydroReservoirs], axis=1)
dataset.interpolate(method = 'linear', limit_direction = 'forward', inplace=True, axis=0)

daylight_features = cf.daylight_extractor(dataset)
holiday_features = cf.get_holidays(dataset)
dataset = cf.calendar_transformer(dataset)

dataset = pd.concat([dataset, daylight_features, holiday_features], axis=1)

dataset.rename(columns = {"spotPrice" : "y"}, inplace=True)
dataset.insert(0, "unique_id", 1)
dataset.insert(0, "ds", dataset.index)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 54408 entries, 2018-01-01 00:00:00 to 2024-03-16 23:00:00
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ds                          54408 non-null  datetime64[ns]
 1   unique_id                   54408 non-null  int64         
 2   y                           54408 non-null  float64       
 3   ConsumptionSE1GWh           54408 non-null  float64       
 4   ConsumptionSE2GWh           54408 non-null  float64       
 5   ConsumptionSE3GWh           54408 non-null  float64       
 6   ConsumptionSE4GWh           54408 non-null  float64       
 7   waterSE1(MWh)               54408 non-null  float64       
 8   windSE1(MWh)                54408 non-null  float64       
 9   trmSE1MWh                   54408 non-null  float64       
 10  waterSE2(MWh)               54408 non-null  float64       
 11  windSE2(MWh)       

In [4]:
import torch

device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Your Device is {device}.")

Your Device is cuda.


In [5]:

test_period = 120
futr_exog_list = dataset.iloc[:, 30:].columns.to_list()
past_cov_list = dataset.iloc[:, 2:30].columns.to_list()

validation_split = int(0.2 * len(dataset.ds.unique()))

train_df = dataset.iloc[:-validation_split - test_period]
val_df = dataset.iloc[-validation_split - test_period:-test_period]
test_df = dataset.iloc[-test_period:]

test_df = test_df.drop(columns=past_cov_list[1:])

h = val_df.ds.nunique()

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import MAE, MSE, MAPE

horizon = 24 * 5

nf = NeuralForecast(
    models = [LSTM(h = h, 
                input_size=-1, 
                
                max_steps= 30,
                scaler_type = "minmax",
                batch_size = 32,
                
                loss=MAE(),
                
                futr_exog_list = futr_exog_list,
                hist_exog_list = past_cov_list,
                
                encoder_n_layers=1, 
                encoder_hidden_size = 16,
                
                context_size = 10, 
                
                decoder_hidden_size = 16,   
                decoder_layers=1, 
                )], freq = "H"
)

nf.fit(df=train_df, verbose = True)

In [ ]:
y_hat = nf.predict(futr_df=test_df)

plt.figure(figsize=(16, 10))
plt.plot(test_df.ds, test_df.y, label = "True")
plt.plot(y_hat.ds, y_hat.LSTM, label = "Predicted", alpha = 0.7)
plt.legend()
plt.show()